# Response Item Network Analysis

## Import

In [ ]:
# Import 
import pandas as pd

# Import custom libraries
import util
import responseItemNetwork
from util import UtilityFunctions as uf
from responseItemNetwork import ResponseItemNetwork as ResIN

# reload due to changes 
from importlib import reload
reload(util)   
reload(responseItemNetwork)  

# Monkey patch the method from the utility class to the pandas DataFrame
pd.DataFrame.filter_features = uf.filter_features
pd.DataFrame.filter_numerical_values = uf.filter_numerical_values
pd.DataFrame.filter_negative_values = uf.filter_negative_values
pd.DataFrame.filter_columns_with_less_unique_values_than_threshold = uf.filter_columns_with_less_unique_values_than_threshold
pd.DataFrame.drop_columns = uf.drop_columns

## Load Data 

In [ ]:
path = './../data/processed/'
file_to_load = "2018_U"
processed_files = uf.get_csv_files_from_folder(path)
for f in processed_files:
        if f.__contains__(file_to_load):
            print(f)
            df = pd.read_csv(f)

## Select Features

In [ ]:
question_mapping = {
    # Economic Views
    'E035': 'Income Equality',     # Income equality vs individual incentives (1-10)
    'E036': 'State Ownership',     # Private vs state ownership (1-10)
    'E037': 'Gov Responsibility',  # Government vs individual responsibility (1-10)
    'E039': 'Competition',         # Competition good vs harmful (1-10)
    
    # Political Trust/System
    'E033': 'Political Scale',     # Self positioning 
    'E069_11': 'Gov Trust',        # Confidence in government
    'E069_12': 'Party Trust',      # Confidence in political parties
    'E069_07': 'Parliament Trust', # Confidence in parliament
    'E114': 'Strong Leader',       # Having a strong leader
    'E116': 'Military Rule',       # Having army rule
    
    # Social Issues
    'F120': 'Abortion',           # Justifiable: Abortion
    'F118': 'Homosexuality',      # Justifiable: Homosexuality
    'F119': 'Prostitution',       # Justifiable: Prostitution
    'F144_02': 'Death Penalty',   # Justifiable: Death penalty
    
    # Immigration/Nationalism
    'G052': 'Immigration Impact',  # Impact of immigrants on country
    #'C002_01': 'Jobs Priority',    # Priority to nationals over immigrants
    
    # Gender/Social Equality
    'D059': 'Women Leaders',      # Men better political leaders
    'D078': 'Women Executives',   # Men better business executives
    'D060': 'University Gender',  # University more important for boys
    
    # Government Power
    'H009': 'Surveillance',       # Government surveillance rights
    'H010': 'Internet Monitor',   # Monitor emails and internet
    'H011': 'Data Collection',    # Collect information without knowledge
    
    # Economic Policy
    'E224': 'Wealth Transfer',    # Tax rich and subsidize poor
    'B008': 'Environment vs Growth', # Environmental protection vs Economic growth
    
    # Traditional Values
    'F034': 'Religious Person',   # Religious person
    'D026_03': 'Child Duty',      # Duty towards society to have children
    'E018': 'Authority Respect'   # Greater respect for authority
}
df_filtered = df.filter_features(list(question_mapping.keys()))
print(df_filtered.columns)

## Mapping features to questions

## Process Features Into Binary variables 

In [ ]:
resin = ResIN(df_filtered)
ResIN.visualize_graph(resin, question_mapping, None, False)
print(resin.df.shape)
print(resin.df.columns)

print((resin.df['E035'] == 1).sum())
print((resin.df['E036'] == 1).sum())
print((resin.df['E033'] == 1).sum())